In [ ]:
!pip install sympy==1.12
import sympy
print(sympy.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 108.7 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.3
    Uninstalling sympy-1.13.3:
      Successfully uninstalled sympy-1.13.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.9.0+cu126 requires sympy>=1.13.3, but you have sympy 1.12 which is incompatible.


1.13.3


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import random

# =========================================================
# 0) Seed 고정
# =========================================================
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

# =========================================================
# 1) 평가 지표
# =========================================================
def smape(a, f):
    a = np.array(a)
    f = np.array(f)
    denom = (np.abs(a) + np.abs(f)) / 2
    mask = denom != 0
    return np.mean(np.abs(a[mask] - f[mask]) / denom[mask]) * 100

def evaluate(actual, pred):
    actual = np.array(actual)
    pred = np.array(pred)
    mae = np.mean(np.abs(actual - pred))
    rmse = np.sqrt(np.mean((actual - pred)**2))
    smape_v = smape(actual, pred)
    return mae, rmse, smape_v

# =========================================================
# 데이터 전처리
# =========================================================
def clean_sales(df):
    df['일매출'] = (
        df['일매출']
        .astype(str)
        .str.replace(",", "")
        .str.replace(" ", "")
        .str.strip()
    )
    df['일매출'] = pd.to_numeric(df['일매출'], errors='coerce').fillna(0)
    return df

def add_date_features(df):
    df['month'] = df['영업일자'].dt.month
    df['day'] = df['영업일자'].dt.day
    df['weekday'] = df['영업일자'].dt.weekday
    df['is_weekend'] = (df['weekday'] >= 5).astype(int)

    df['open_hours'] = 11
    df.loc[df['weekday'] == 5, 'open_hours'] = 6
    df.loc[df['weekday'] == 6, 'open_hours'] = 0
    return df

def add_lag_features(df):
    df['lag1'] = df['일매출'].shift(1)
    df['lag7'] = df['일매출'].shift(7)
    df['lag14'] = df['일매출'].shift(14)
    df['lag28'] = df['일매출'].shift(28)

    df['roll_mean7'] = df['일매출'].rolling(7).mean()
    df['roll_mean14'] = df['일매출'].rolling(14).mean()
    df['roll_mean28'] = df['일매출'].rolling(28).mean()

    df['roll_std7'] = df['일매출'].rolling(7).std()
    df['roll_std28'] = df['일매출'].rolling(28).std()
    return df

# =========================================================
# Sliding Window
# =========================================================
def create_sequences(df, feature_cols, seq_len=28):
    X, y = [], []
    values = df[feature_cols].values
    targets = df['일매출_log'].values   # ← log-space
    for i in range(seq_len, len(df)):
        X.append(values[i-seq_len:i])
        y.append(targets[i])
    return np.array(X), np.array(y)

# =========================================================
# Dataset
# =========================================================
class SequenceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# =========================================================
# ⭐⭐ 최적화된 GRU 모델 (log-space 학습) ⭐⭐
# =========================================================
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=2):
        super().__init__()
        self.gru = nn.GRU(
            input_dim,
            hidden_dim,
            num_layers=num_layers,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.gru(x)
        return self.fc(out[:, -1, :])   # log-space 출력

# =========================================================
# Autoregressive 예측 (log-space)
# =========================================================
def autoregressive_forecast(model, df_train, df_test, seq_len, feature_cols, scaler_X, scaler_y):

    history = df_train.copy()
    preds = []

    for i in range(len(df_test)):
        row = df_test.iloc[i].copy()

        # 최근 raw 매출 (log-space → real-space)
        last_real = np.expm1(history['일매출_log'].values)

        # ---- lag 업데이트 ----
        row['lag1']  = last_real[-1]
        row['lag7']  = last_real[-7]
        row['lag14'] = last_real[-14]
        row['lag28'] = last_real[-28]

        # rolling
        row['roll_mean7']  = pd.Series(last_real[-7:]).mean()
        row['roll_mean14'] = pd.Series(last_real[-14:]).mean()
        row['roll_mean28'] = pd.Series(last_real[-28:]).mean()

        row['roll_std7']   = pd.Series(last_real[-7:]).std()
        row['roll_std28']  = pd.Series(last_real[-28:]).std()

        # 시퀀스 구성 (feature scaling)
        seq_df = history.tail(seq_len).copy()
        seq_scaled = scaler_X.transform(seq_df[feature_cols])

        X = torch.tensor(seq_scaled, dtype=torch.float32).unsqueeze(0)

        # -------- GRU log 예측 --------
        pred_log_scaled = model(X).item()
        pred_log = scaler_y.inverse_transform([[pred_log_scaled]])[0][0]

        # -------- log → real-space 복원 --------
        pred_real = np.expm1(pred_log)
        pred_real = max(pred_real, 0)

        preds.append(pred_real)

        # -------- history 업데이트: log-space로 저장 --------
        row['일매출_log'] = pred_log
        history = pd.concat([history, pd.DataFrame([row])], ignore_index=True)

    return preds

# =========================================================
# 🔥 메인 실행
# =========================================================
train = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_train_val.csv')
test = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_test.csv')

train['영업일자'] = pd.to_datetime(train['영업일자'])
test['영업일자']  = pd.to_datetime(test['영업일자'])

train = clean_sales(train)
test  = clean_sales(test)

# 학사일정 merge
academic = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/학사일정_정리(2325).csv')
academic['영업일자'] = pd.to_datetime(academic['date'])
academic = academic.drop(columns=['date'])

weekday_map = {'mon':0,'tue':1,'wed':2,'thu':3,'fri':4,'sat':5,'sun':6}
academic['acad_weekday'] = academic['weekday'].map(weekday_map)
academic = academic.drop(columns=['weekday'])

academic = academic.rename(columns={
    'weekend':'acad_weekend',
    'holiday':'acad_holiday',
    'semester':'acad_semester',
    'seasonal':'acad_seasonal',
    'exam':'acad_exam',
    'ceremony':'acad_ceremony',
    'dormitory':'acad_dormitory'
})

train = train.merge(academic, on='영업일자', how='left')
test  = test.merge(academic, on='영업일자',  how='left')

train = add_date_features(train)
test  = add_date_features(test)

train = add_lag_features(train).dropna().reset_index(drop=True)
test  = add_lag_features(test)

# =========================================================
# Feature 15개 (GRU 전용)
# =========================================================
feature_cols = [
    'acad_weekend', 'acad_semester', 'acad_weekday',
    'open_hours', 'acad_ceremony', 'acad_exam',
    'lag1','lag7','lag14','lag28',
    'roll_std7','roll_std28',
    'roll_mean7','roll_mean14','roll_mean28'
]

# =========================================================
# log-space 타깃 생성
# =========================================================
train['일매출_log'] = np.log1p(train['일매출'])
test['일매출_log']  = np.log1p(test['일매출'])

# =========================================================
# Scaling (feature + log-target)
# =========================================================
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

train[feature_cols] = scaler_X.fit_transform(train[feature_cols])
train[['일매출_log']] = scaler_y.fit_transform(train[['일매출_log']])

# =========================================================
# GRU 학습
# =========================================================
seq_len = 28
X_train, y_train = create_sequences(train, feature_cols, seq_len)

dataset = SequenceDataset(X_train, y_train)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

model = GRUModel(input_dim=len(feature_cols), num_layers=2)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(50):
    total_loss = 0
    for Xb, yb in loader:
        optimizer.zero_grad()
        loss = criterion(model(Xb), yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"[Epoch {epoch+1}/50] Loss: {total_loss:.4f}")

# =========================================================
# Test 예측 (log-space → real)
# =========================================================
test_pred = autoregressive_forecast(model, train, test, seq_len, feature_cols, scaler_X, scaler_y)

mae, rmse, smape_val = evaluate(test['일매출'], test_pred)

print("\n====== 최종 성능 (Optimized GRU / log-space) ======")
print("MAE :", mae)
print("RMSE:", rmse)
print("SMAPE:", smape_val)


test['예측매출'] = test_pred
print(test[['영업일자', '예측매출']])

test[['영업일자', '예측매출']].to_csv("prediction_gru_logspace.csv", index=False)

print("\n📌 저장 완료 → prediction_gru_logspace.csv")


[Epoch 1/50] Loss: 5.1248
[Epoch 2/50] Loss: 3.4537
[Epoch 3/50] Loss: 3.0589
[Epoch 4/50] Loss: 2.8459
[Epoch 5/50] Loss: 2.5098
[Epoch 6/50] Loss: 2.3306
[Epoch 7/50] Loss: 2.2322
[Epoch 8/50] Loss: 2.0843
[Epoch 9/50] Loss: 1.7841
[Epoch 10/50] Loss: 1.6079
[Epoch 11/50] Loss: 1.5385
[Epoch 12/50] Loss: 1.4832
[Epoch 13/50] Loss: 1.4270
[Epoch 14/50] Loss: 1.4800
[Epoch 15/50] Loss: 1.4239
[Epoch 16/50] Loss: 1.5282
[Epoch 17/50] Loss: 1.4763
[Epoch 18/50] Loss: 1.4326
[Epoch 19/50] Loss: 1.4526
[Epoch 20/50] Loss: 1.3838
[Epoch 21/50] Loss: 1.3701
[Epoch 22/50] Loss: 1.4209
[Epoch 23/50] Loss: 1.3956
[Epoch 24/50] Loss: 1.4222
[Epoch 25/50] Loss: 1.3865
[Epoch 26/50] Loss: 1.3789
[Epoch 27/50] Loss: 1.3619
[Epoch 28/50] Loss: 1.3619
[Epoch 29/50] Loss: 1.4993
[Epoch 30/50] Loss: 1.3531
[Epoch 31/50] Loss: 1.3833
[Epoch 32/50] Loss: 1.3628
[Epoch 33/50] Loss: 1.4028
[Epoch 34/50] Loss: 1.3772
[Epoch 35/50] Loss: 1.3588
[Epoch 36/50] Loss: 1.4113
[Epoch 37/50] Loss: 1.2954
[Epoch 38/

**성능 안정화**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import random

# =========================================================
# 0) Seed 고정
# =========================================================
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

# =========================================================
# 1) 평가 지표
# =========================================================
def smape(a, f):
    a = np.array(a)
    f = np.array(f)
    denom = (np.abs(a) + np.abs(f)) / 2
    mask = denom != 0
    return np.mean(np.abs(a[mask] - f[mask]) / denom[mask]) * 100

def evaluate(actual, pred):
    actual = np.array(actual)
    pred = np.array(pred)
    mae = np.mean(np.abs(actual - pred))
    rmse = np.sqrt(np.mean((actual - pred)**2))
    smape_v = smape(actual, pred)
    return mae, rmse, smape_v

# =========================================================
# 데이터 전처리
# =========================================================
def clean_sales(df):
    df['일매출'] = (
        df['일매출']
        .astype(str)
        .str.replace(",", "")
        .str.replace(" ", "")
        .str.strip()
    )
    df['일매출'] = pd.to_numeric(df['일매출'], errors='coerce').fillna(0)
    return df

def add_date_features(df):
    df['month'] = df['영업일자'].dt.month
    df['day'] = df['영업일자'].dt.day
    df['weekday'] = df['영업일자'].dt.weekday
    df['is_weekend'] = (df['weekday'] >= 5).astype(int)

    df['open_hours'] = 11
    df.loc[df['weekday'] == 5, 'open_hours'] = 6
    df.loc[df['weekday'] == 6, 'open_hours'] = 0
    return df

def add_lag_features(df):
    df['lag1'] = df['일매출'].shift(1)
    df['lag7'] = df['일매출'].shift(7)
    df['lag14'] = df['일매출'].shift(14)
    df['lag28'] = df['일매출'].shift(28)

    df['roll_mean7'] = df['일매출'].rolling(7).mean()
    df['roll_mean14'] = df['일매출'].rolling(14).mean()
    df['roll_mean28'] = df['일매출'].rolling(28).mean()

    df['roll_std7'] = df['일매출'].rolling(7).std()
    df['roll_std28'] = df['일매출'].rolling(28).std()
    return df

# =========================================================
# Sliding Window
# =========================================================
def create_sequences(df, feature_cols, seq_len=28):
    X, y = [], []
    values = df[feature_cols].values
    targets = df['일매출_log'].values   # log-space target
    for i in range(seq_len, len(df)):
        X.append(values[i-seq_len:i])
        y.append(targets[i])
    return np.array(X), np.array(y)

# =========================================================
# Dataset
# =========================================================
class SequenceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# =========================================================
# ⭐⭐ 안정 버전 GRU 모델 ⭐⭐
# =========================================================
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=96, num_layers=2, dropout=0.2):
        super().__init__()
        self.gru = nn.GRU(
            input_dim,
            hidden_dim,
            num_layers=num_layers,
            dropout=dropout,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.gru(x)
        return self.fc(out[:, -1, :])   # log-space 출력

# =========================================================
# Autoregressive 예측 (log-space 안정 버전)
# =========================================================
def autoregressive_forecast(model, train_df, test_df, seq_len, feature_cols, scaler_X):

    # 초기 history는 train에서 “seq_len 이후 구간” 사용 → lag/rolling 100% 정상
    history_real = train_df['일매출'].tolist()
    history_log  = train_df['일매출_log'].tolist()

    preds = []

    for i in range(len(test_df)):
        row = test_df.iloc[i].copy()

        # 현재까지 real-history 배열
        arr = np.array(history_real)

        # -------- lag 계산 (안전 모드) --------
        def safe_lag(k):
            return arr[-k] if len(arr) >= k else arr[0]

        row['lag1']  = safe_lag(1)
        row['lag7']  = safe_lag(7)
        row['lag14'] = safe_lag(14)
        row['lag28'] = safe_lag(28)

        # -------- rolling --------
        def safe_roll(k):
            if len(arr) >= k:
                return np.mean(arr[-k:])
            else:
                return np.mean(arr)

        def safe_std(k):
            if len(arr) >= k:
                return np.std(arr[-k:])
            else:
                return 0.0

        row['roll_mean7']  = safe_roll(7)
        row['roll_mean14'] = safe_roll(14)
        row['roll_mean28'] = safe_roll(28)

        row['roll_std7']   = safe_std(7)
        row['roll_std28']  = safe_std(28)

        # ===== seq_df 생성 =====
        # 항상 train_df 기반 (sequence 안정)
        seq_df = train_df.tail(seq_len).copy()

        # feature scaling
        seq_scaled = scaler_X.transform(seq_df[feature_cols])

        X = torch.tensor(seq_scaled, dtype=torch.float32).unsqueeze(0)

        # -------- GRU log 예측 --------
        pred_log = model(X).item()

        # 안정화
        pred_log = np.clip(pred_log, -3, 12)

        # log → real
        pred_real = np.expm1(pred_log)
        pred_real = max(pred_real, 0)

        preds.append(pred_real)

        # history 업데이트
        history_real.append(pred_real)
        history_log.append(pred_log)

    return preds



# =========================================================
# 🔥 메인 실행
# =========================================================
train = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_train_val.csv')
test  = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_test.csv')

train['영업일자'] = pd.to_datetime(train['영업일자'])
test['영업일자']  = pd.to_datetime(test['영업일자'])

train = clean_sales(train)
test  = clean_sales(test)

# 학사일정 merge
academic = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/학사일정_정리(2325).csv')
academic['영업일자'] = pd.to_datetime(academic['date'])
academic = academic.drop(columns=['date'])

weekday_map = {'mon':0,'tue':1,'wed':2,'thu':3,'fri':4,'sat':5,'sun':6}
academic['acad_weekday'] = academic['weekday'].map(weekday_map)
academic = academic.drop(columns=['weekday'])

academic = academic.rename(columns={
    'weekend':'acad_weekend',
    'holiday':'acad_holiday',
    'semester':'acad_semester',
    'seasonal':'acad_seasonal',
    'exam':'acad_exam',
    'ceremony':'acad_ceremony',
    'dormitory':'acad_dormitory'
})

train = train.merge(academic, on='영업일자', how='left')
test  = test.merge(academic, on='영업일자', how='left')

train = add_date_features(train)
test  = add_date_features(test)

train = add_lag_features(train).dropna().reset_index(drop=True)
test  = add_lag_features(test)

# =========================================================
# Feature 15개
# =========================================================
feature_cols = [
    'acad_weekend', 'acad_semester', 'acad_weekday',
    'open_hours', 'acad_ceremony', 'acad_exam',
    'lag1','lag7','lag14','lag28',
    'roll_std7','roll_std28',
    'roll_mean7','roll_mean14','roll_mean28'
]

# =========================================================
# log-space 타깃
# =========================================================
train['일매출_log'] = np.log1p(train['일매출'])
test['일매출_log']  = np.log1p(test['일매출'])

# =========================================================
# Feature Scaling
# =========================================================
scaler_X = MinMaxScaler()
train[feature_cols] = scaler_X.fit_transform(train[feature_cols])

# =========================================================
# GRU 학습
# =========================================================
seq_len = 28
X_train, y_train = create_sequences(train, feature_cols, seq_len)

dataset = SequenceDataset(X_train, y_train)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

model = GRUModel(input_dim=len(feature_cols), num_layers=2)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(50):
    total_loss = 0
    for Xb, yb in loader:
        optimizer.zero_grad()
        loss = criterion(model(Xb), yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"[Epoch {epoch+1}/50] Loss: {total_loss:.4f}")

# =========================================================
# Test 예측 (log → real)
# =========================================================
test_pred = autoregressive_forecast(model, train, test, seq_len, feature_cols, scaler_X)

mae, rmse, smape_val = evaluate(test['일매출'], test_pred)

print("\n====== 최종 성능 (Stable GRU / log-space) ======")
print("MAE :", mae)
print("RMSE:", rmse)
print("SMAPE:", smape_val)

test['예측매출'] = test_pred
print(test[['영업일자', '예측매출']])

test[['영업일자', '예측매출']].to_csv("prediction_gru_stable.csv", index=False)

print("\n📌 저장 완료 → prediction_gru_stable.csv")


[Epoch 1/50] Loss: 1765.0869
[Epoch 2/50] Loss: 747.7218
[Epoch 3/50] Loss: 660.6371
[Epoch 4/50] Loss: 645.6322
[Epoch 5/50] Loss: 640.5663
[Epoch 6/50] Loss: 636.4551
[Epoch 7/50] Loss: 652.1385
[Epoch 8/50] Loss: 640.7224
[Epoch 9/50] Loss: 649.1341
[Epoch 10/50] Loss: 644.4420
[Epoch 11/50] Loss: 635.8274
[Epoch 12/50] Loss: 644.2869
[Epoch 13/50] Loss: 653.1559
[Epoch 14/50] Loss: 645.7050
[Epoch 15/50] Loss: 648.0791
[Epoch 16/50] Loss: 650.4869
[Epoch 17/50] Loss: 648.5586
[Epoch 18/50] Loss: 648.6203
[Epoch 19/50] Loss: 649.4565
[Epoch 20/50] Loss: 647.7044
[Epoch 21/50] Loss: 647.6448
[Epoch 22/50] Loss: 637.8861
[Epoch 23/50] Loss: 649.3445
[Epoch 24/50] Loss: 653.6799
[Epoch 25/50] Loss: 633.6795
[Epoch 26/50] Loss: 652.5087
[Epoch 27/50] Loss: 632.0541
[Epoch 28/50] Loss: 641.6149
[Epoch 29/50] Loss: 649.3116
[Epoch 30/50] Loss: 637.9044
[Epoch 31/50] Loss: 641.0362
[Epoch 32/50] Loss: 640.7006
[Epoch 33/50] Loss: 648.7731
[Epoch 34/50] Loss: 632.2747
[Epoch 35/50] Loss: 63